In [1]:
%load_ext autoreload

In [2]:
%aimport model

In [3]:
%aimport data

Files already downloaded and verified
Files already downloaded and verified


In [4]:
import skorch

In [5]:
import numpy as np

In [6]:
import keras

Using TensorFlow backend.


In [7]:
import torch

In [8]:
dataset = "mnist"

In [9]:
dataset_config = data.DATASET_CONFIGS[dataset]

In [10]:
total_block_number = 6

In [11]:
widen_factor = 10

In [12]:
wrn = model.WideResNet(
    dataset,
    dataset_config['size'],
    dataset_config['channels'],
    dataset_config['classes'],
    total_block_number=total_block_number,
    widen_factor=widen_factor,
)

In [13]:
wrn.cuda()

WideResNet(
  (sequential): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): ResidualBlockGroup(
      (sequential): Sequential(
        (0): ResidualBlock(
          (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu1): ReLU(inplace)
          (conv1): Conv2d(16, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu2): ReLU(inplace)
          (conv2): Conv2d(160, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (conv_transform): Conv2d(16, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
        )
        (1): ResidualBlock(
          (bn1): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu1): ReLU(inplace)
          (conv1): Conv2d(160, 160, kernel_size=(3, 3), stride=(1, 

In [14]:
net = skorch.net.NeuralNetClassifier(
    module=wrn,
    max_epochs=10,
    optimizer=torch.optim.Adam,
    callbacks=[
        ('checkpoint', skorch.callbacks.Checkpoint(target="models/wrn{last_epoch[epoch]}.pt", monitor=None))
    ],
)
net.initialize()

<class 'skorch.net.NeuralNetClassifier'>[initialized](
  module_=WideResNet(
    (sequential): Sequential(
      (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): ResidualBlockGroup(
        (sequential): Sequential(
          (0): ResidualBlock(
            (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu1): ReLU(inplace)
            (conv1): Conv2d(16, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn2): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu2): ReLU(inplace)
            (conv2): Conv2d(160, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (conv_transform): Conv2d(16, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
          )
          (1): ResidualBlock(
            (bn1): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     

In [15]:
def process(images):
    pad_images = np.pad(images, ((0, 0), (2, 2), (2, 2)), 'constant')
    number_of_images, height, width = pad_images.shape
    return pad_images.reshape(number_of_images, 1, height, width).astype(np.float32)  / 255.0

In [16]:
(x_train_bytes, y_train_bytes), (x_test_bytes, y_test_bytes) = keras.datasets.fashion_mnist.load_data()
x_train = process(x_train_bytes)
x_test = process(x_test_bytes)
y_train = y_train_bytes.astype(np.int64)
y_test = y_test_bytes.astype(np.int64)

In [ ]:
net.fit_loop(x_train, y_train)